In [1]:
import pandas as pd

In [2]:
movies_df = pd.read_csv('data/ratings_movies.csv')
print(movies_df.head())

   Unnamed: 0  userId  movieId  rating                 date  \
0           0       1        1     4.0  2000-07-30 18:45:03   
1           1       1        3     4.0  2000-07-30 18:20:47   
2           2       1        6     4.0  2000-07-30 18:37:04   
3           3       1       47     5.0  2000-07-30 19:03:35   
4           4       1       50     5.0  2000-07-30 18:48:51   

                         title                                       genres  
0             Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy  
1      Grumpier Old Men (1995)                               Comedy|Romance  
2                  Heat (1995)                        Action|Crime|Thriller  
3  Seven (a.k.a. Se7en) (1995)                             Mystery|Thriller  
4   Usual Suspects, The (1995)                       Crime|Mystery|Thriller  


In [3]:
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [4]:
movies_df['year_release']=movies_df['title'].apply(get_year_release)
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    100836 non-null  int64  
 1   userId        100836 non-null  int64  
 2   movieId       100836 non-null  int64  
 3   rating        100836 non-null  float64
 4   date          100836 non-null  object 
 5   title         100836 non-null  object 
 6   genres        100836 non-null  object 
 7   year_release  100818 non-null  float64
dtypes: float64(2), int64(3), object(3)
memory usage: 6.2+ MB


In [5]:
filtered = movies_df[movies_df['year_release']==1999]
filtered.groupby('title')['rating'].mean().sort_values()

title
Bloodsport: The Dark Kumite (1999)            0.5
Simon Sez (1999)                              1.0
Chill Factor (1999)                           1.0
Source, The (1999)                            1.0
Trippin' (1999)                               1.0
                                             ... 
Trailer Park Boys (1999)                      5.0
Larry David: Curb Your Enthusiasm (1999)      5.0
Sun Alley (Sonnenallee) (1999)                5.0
George Carlin: You Are All Diseased (1999)    5.0
Five Senses, The (1999)                       5.0
Name: rating, Length: 261, dtype: float64

In [6]:
mask = movies_df['year_release']==2010
movies_df [mask].groupby('genres')['rating'].mean().sort_values()

genres
Action|Sci-Fi                        1.000000
Action|Adventure|Horror              1.500000
Action|Drama|Fantasy                 1.500000
Crime|Romance                        1.500000
Adventure|Comedy|Fantasy             1.833333
                                       ...   
Crime                                4.750000
Comedy|Musical                       5.000000
Animation|Drama|Fantasy|Mystery      5.000000
Adventure|Children|Comedy|Mystery    5.000000
Animation|Children|Mystery           5.000000
Name: rating, Length: 119, dtype: float64

In [7]:
movies_df.groupby('userId')['genres'].nunique().sort_values(ascending=False)

userId
599    524
414    482
448    403
380    399
474    395
      ... 
578     15
12      15
85      13
214     13
245     13
Name: genres, Length: 610, dtype: int64

In [8]:
movies_df.groupby('userId')['rating'].agg(['count', 'mean']).sort_values(by = ['count', 'mean'], ascending = [True, False])

,count,mean
userId,,
53,20,5.000000
595,20,4.200000
189,20,4.100000
569,20,4.000000
278,20,3.875000
...,...,...
274,1346,3.235884
448,1864,2.847371
474,2108,3.398956


In [9]:
mask = movies_df['year_release'] == 2018
grouped = movies_df[mask].groupby('genres')['rating'].agg(
    ['mean', 'count']
)
grouped[grouped['count']>10].sort_values(
    by='mean',
    ascending=False
)

,mean,count
genres,,
Action|Adventure|Sci-Fi,3.928571,14
Action|Comedy|Sci-Fi,3.875000,12


In [10]:
movies_df['date']=pd.to_datetime(movies_df['date'])
movies_df['year_rating']=movies_df['date'].dt.year
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Unnamed: 0    100836 non-null  int64         
 1   userId        100836 non-null  int64         
 2   movieId       100836 non-null  int64         
 3   rating        100836 non-null  float64       
 4   date          100836 non-null  datetime64[ns]
 5   title         100836 non-null  object        
 6   genres        100836 non-null  object        
 7   year_release  100818 non-null  float64       
 8   year_rating   100836 non-null  int32         
dtypes: datetime64[ns](1), float64(2), int32(1), int64(3), object(2)
memory usage: 6.5+ MB


In [11]:
pivot = movies_df.pivot_table(index='year_rating', columns='genres', values='rating',aggfunc='mean')
print(pivot)

genres       (no genres listed)    Action  Action|Adventure  \
year_rating                                                   
1996                        NaN  2.730769          3.454545   
1997                        NaN  3.538462          4.150000   
1998                        NaN       NaN          4.200000   
1999                        NaN       NaN          4.000000   
2000                        NaN  2.588235          3.738462   
2001                        NaN  3.000000          3.500000   
2002                        NaN  2.750000          4.304348   
2003                        NaN  3.833333          3.277778   
2004                        NaN  2.700000          4.136364   
2005                        NaN  3.357143          3.413043   
2006                        NaN  2.166667          4.090909   
2007                        NaN  2.583333          3.666667   
2008                        NaN  3.000000          3.656250   
2009                        NaN       NaN          3.71

In [12]:
print(pivot['Action|Adventure'])

year_rating
1996    3.454545
1997    4.150000
1998    4.200000
1999    4.000000
2000    3.738462
2001    3.500000
2002    4.304348
2003    3.277778
2004    4.136364
2005    3.413043
2006    4.090909
2007    3.666667
2008    3.656250
2009    3.714286
2010    3.437500
2011    3.888889
2012    3.900000
2013    3.833333
2014    3.875000
2015    3.560976
2016    3.722222
2017    3.527778
2018    3.397436
Name: Action|Adventure, dtype: float64


In [13]:
print(pivot['Action|Adventure|Animation|Children|Comedy|IMAX'].sort_values(ascending=False))

year_rating
2013    5.000000
2018    3.700000
2016    3.666667
2012    3.550000
2010    3.500000
2014    3.500000
2015    3.375000
2009    3.250000
2017    1.000000
1996         NaN
1997         NaN
1998         NaN
1999         NaN
2000         NaN
2001         NaN
2002         NaN
2003         NaN
2004         NaN
2005         NaN
2006         NaN
2007         NaN
2008         NaN
2011         NaN
Name: Action|Adventure|Animation|Children|Comedy|IMAX, dtype: float64


In [14]:
print(pivot['Comedy'])

year_rating
1996    3.228571
1997    3.409091
1998    3.000000
1999    3.606061
2000    3.141291
2001    3.318408
2002    3.198556
2003    3.120066
2004    3.356877
2005    2.963325
2006    3.347534
2007    2.928187
2008    3.426667
2009    3.273292
2010    3.179825
2011    3.232877
2012    3.478477
2013    3.489474
2014    3.271429
2015    3.106183
2016    3.373431
2017    2.852668
2018    3.121296
Name: Comedy, dtype: float64


In [15]:
print(pivot['Animation|Children|Mystery'])

year_rating
1996    NaN
1997    NaN
1998    NaN
1999    NaN
2000    NaN
2001    NaN
2002    NaN
2003    NaN
2004    NaN
2005    NaN
2006    NaN
2007    NaN
2008    NaN
2009    NaN
2010    NaN
2011    NaN
2012    NaN
2013    NaN
2014    NaN
2015    NaN
2016    NaN
2017    NaN
2018    5.0
Name: Animation|Children|Mystery, dtype: float64


In [16]:
mask = movies_df['year_release'] == 2018

In [21]:
print(pivot.loc[2018])

genres
(no genres listed)                     3.676471
Action                                 2.588235
Action|Adventure                       3.397436
Action|Adventure|Animation             3.650000
Action|Adventure|Animation|Children    3.166667
                                         ...   
Sci-Fi|Thriller                        3.562500
Sci-Fi|Thriller|IMAX                        NaN
Thriller                               3.296296
War                                         NaN
Western                                3.611111
Name: 2018, Length: 951, dtype: float64
